<a href="https://colab.research.google.com/github/nalgo-intern/team-a/blob/master/sample_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 2.8MB/s 


In [0]:
downloaded = drive.CreateFile({'id':'1UAlxDtZompZuoqEI7hfdN47gGgA6RW5E'})

downloaded.GetContentFile('train.zip')

In [0]:
downloaded = drive.CreateFile({'id':'1xhHhaH92EIfwZ0lNGO_VFLiWQL7qvBV7'})

downloaded.GetContentFile('validation.zip')

In [0]:
!unzip train.zip
!unzip validation.zip

Archive:  train.zip
   creating: train/chimpanzee/
  inflating: train/chimpanzee/000001.jpg  
  inflating: train/chimpanzee/000002.jpg  
  inflating: train/chimpanzee/000003.jpg  
  inflating: train/chimpanzee/000006.jpg  
  inflating: train/chimpanzee/000007.jpg  
  inflating: train/chimpanzee/000009.jpg  
  inflating: train/chimpanzee/000010.jpg  
  inflating: train/chimpanzee/000011.jpg  
  inflating: train/chimpanzee/000012.jpg  
  inflating: train/chimpanzee/000013.jpg  
  inflating: train/chimpanzee/000014.jpg  
  inflating: train/chimpanzee/000015.jpg  
  inflating: train/chimpanzee/000016.jpg  
  inflating: train/chimpanzee/000017.jpg  
  inflating: train/chimpanzee/000019.jpg  
  inflating: train/chimpanzee/000020.jpg  
  inflating: train/chimpanzee/000021.jpg  
  inflating: train/chimpanzee/000022.jpg  
  inflating: train/chimpanzee/000023.png  
  inflating: train/chimpanzee/000024.jpg  
  inflating: train/chimpanzee/000025.JPG  
  inflating: train/chimpanzee/000027.jpg  
  i

In [0]:
from keras.models import Model
from keras.layers import Dense,Input,GlobalMaxPooling2D,Dropout
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD

N_CATEGORIES = 3
IMAGE_SIZE = 224
BATCH_SIZE = 8

NUM_TRAINING = 288
NUM_VALIDATION = 72

input_tensor = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
base_model = VGG16(weights='imagenet', include_top=False,input_tensor=input_tensor)

x = base_model.output
x = GlobalMaxPooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(.25)(x)
x = Dense(1024, activation='relu')(x)


predictions = Dense(N_CATEGORIES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers[:15]:
   layer.trainable = False

model.compile(optimizer=SGD(lr=1e-4, momentum=0.9), loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

json_string=model.to_json()
open("model"+'.json','w').write(json_string)

train_datagen = ImageDataGenerator(
   rescale=1.0 / 255,
   shear_range=0,
   zoom_range=0.1,
   horizontal_flip=True,
   rotation_range=0)

test_datagen = ImageDataGenerator(
   rescale=1.0 / 255,
)
train_generator = train_datagen.flow_from_directory(
   'train',
   target_size=(IMAGE_SIZE, IMAGE_SIZE),
   batch_size=BATCH_SIZE,
   class_mode='categorical',
   shuffle=True
)

validation_generator = test_datagen.flow_from_directory(
   'validation',
   target_size=(IMAGE_SIZE, IMAGE_SIZE),
   batch_size=BATCH_SIZE,
   class_mode='categorical',
   shuffle=True
)

hist = model.fit_generator(train_generator,
   steps_per_epoch=NUM_TRAINING//BATCH_SIZE,
   epochs=40,
   verbose=1,
   validation_data=validation_generator,
   validation_steps=NUM_VALIDATION//BATCH_SIZE,
   )

model.save('monkey.hdf5')

Using TensorFlow backend.
W0821 05:52:37.976657 140353782814592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0821 05:52:38.019465 140353782814592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0821 05:52:38.028425 140353782814592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0821 05:52:38.090181 140353782814592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



58892288/58889256 [==============================] - 6s 0us/step


W0821 05:52:45.650279 140353782814592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0821 05:52:45.651520 140353782814592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0821 05:52:49.360656 140353782814592 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0821 05:52:49.430140 140353782814592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:7

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

W0821 05:52:49.788503 140353782814592 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/40
36/36 [==============================] - 16s 447ms/step - loss: 1.1141 - acc: 0.3542 - val_loss: 1.0856 - val_acc: 0.3611
Epoch 2/40
36/36 [==============================] - 14s 387ms/step - loss: 1.0724 - acc: 0.4306 - val_loss: 1.0611 - val_acc: 0.4444
Epoch 3/40
36/36 [==============================] - 14s 378ms/step - loss: 1.0420 - acc: 0.4653 - val_loss: 1.0396 - val_acc: 0.5139
Epoch 4/40
36/36 [==============================] - 14s 391ms/step - loss: 1.0013 - acc: 0.5347 - val_loss: 1.0039 - val_acc: 0.5556
Epoch 5/40
36/36 [==============================] - 14s 383ms/step - loss: 0.9715 - acc: 0.5382 - val_loss: 0.9721 - val_acc: 0.5972
Epoch 6/40
36/36 [==============================] - 14s 379ms/step - loss: 0.8999 - acc: 0.6771 - val_loss: 0.9329 - val_acc: 0.5972
Epoch 7/40
36/36 [==============================] - 14s 393ms/step - loss: 0.8512 - acc: 0.6458 - val_loss: 0.9157 - val_acc: 0.5694
Epoch 8/40
36/36 [==============================] - 14s 396ms/step - 

In [0]:
upload_file_2 = drive.CreateFile()
upload_file_2.SetContentFile("monkey.hdf5")
upload_file_2.Upload()

In [0]:
from google.colab import files
files.download('model.json')

In [17]:
from keras.models import Model
from keras.layers import Dense, GlobalMaxPooling2D,Input,Dropout
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD

import matplotlib.pyplot as plt
import os,random
from keras.preprocessing.image import img_to_array, load_img

file_name='monkey'
display_dir='キングコング.jpg'
label=['chimpanzee','gorilla','orangutan']

N_CATEGORIES  = 3
IMAGE_SIZE = 224
BATCH_SIZE = 8
input_tensor = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
base_model = VGG16(weights='imagenet', include_top=False,input_tensor=input_tensor)
x = base_model.output
x = GlobalMaxPooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(.25)(x)
x = Dense(1024, activation='relu')(x)

predictions = Dense(N_CATEGORIES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

model.load_weights(file_name+'.hdf5')

model.compile(optimizer=SGD(lr=1e-4,momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


files=os.listdir(display_dir)


plt.figure(figsize=(10,10))
for i in range(filenumber):
    temp_img=load_img(os.path.join(display_dir,img[i]),target_size=(224,224))
    plt.subplot(5,5,i+1)
    plt.imshow(temp_img)
    
    temp_img_array=img_to_array(temp_img)
    temp_img_array=temp_img_array.astype('float32')/255.0
    temp_img_array=temp_img_array.reshape((1,224,224,3))
    
    img_pred=model.predict(temp_img_array)
    plt.title(label[np.argmax(img_pred)])
    
    plt.xticks([]),plt.yticks([])

plt.show()

FileNotFoundError: ignored